In [ ]:
!pip install python-dotenv


In [ ]:
https://colab.research.google.com/drive/1X0igIfV5b6HVFfSMQhXJg5Pw7bjpVA5q#scrollTo=a7j_FgeJcr1T

In [ ]:
# I hate python warnings
import warnings
warnings.filterwarnings('ignore')

# In order to load environment variables, import these 2 libraries:
import os
from dotenv import load_dotenv

# Load your environment variables
load_dotenv('.env')  # Load environment variables from the file '.env'
#API_KEY_FMP = '8fcb0de28daf241dd4522f0f455f2636'
API_KEY_FMP = os.environ.get('API_KEY_FMP')  # Retrieve the value of the environment variable 'API_KEY_FMP'
API_KEY_FRED = os.environ.get('API_KEY_FRED')

In [ ]:

import pandas as pd

link = (
    "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies#S&P_500_component_stocks"
)
df = pd.read_html(link, header=0)[0]

In [ ]:
df

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Biotechnology,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989
...,...,...,...,...,...,...,...,...
498,XYL,Xylem Inc.,Industrials,Industrial Machinery & Supplies & Components,"White Plains, New York",2011-11-01,1524472,2011
499,YUM,Yum! Brands,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,1997
500,ZBRA,Zebra Technologies,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,877212,1969
501,ZBH,Zimmer Biomet,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1136869,1927


In [ ]:
tick_dict = dict(zip(df.Symbol, df.Security))

In [ ]:
#tick_dict

In [ ]:
tick_list = list(df.Symbol)

In [ ]:
i = 0
df = pd.DataFrame()
import requests
for i in range(len(tick_list)):
  # Construct your API request endpoint url
  BASE_URL = 'https://financialmodelingprep.com/api/v3'
  company_tick = tick_list[i]
  endpoint_url_dividend = f"{BASE_URL}/historical-price-full/stock_dividend/{company_tick}?apikey={API_KEY_FMP}"
  # Send an HTTP GET request to the endpoint URL and store the response
  response = requests.get(endpoint_url_dividend)
  if response.status_code == 429:
    print("FMP API limit reached")
    # Convert json to dictionary object and then a Pandas Dataframe
    import pandas as pd
    response_dict = response.json()
    dividends = pd.DataFrame(response_dict['historical'])
    df_test = df_test.append(dividends)

In [ ]:
df_test

NameError: name 'df_test' is not defined

In [ ]:
# Import the requests library
import requests


https://github.com/datasets/s-and-p-500-companies/blob/main/data/constituents.csv

# Construct your API request endpoint url
BASE_URL = 'https://financialmodelingprep.com/api/v3'
company_tick = "AAPL"
endpoint_url_dividend = f"{BASE_URL}/historical-price-full/stock_dividend/{company_tick}?apikey={API_KEY_FMP}"


# Send an HTTP GET request to the endpoint URL and store the response
response = requests.get(endpoint_url_dividend)
if response.status_code == 429:
    print("FMP API limit reached")


# Convert json to dictionary object and then a Pandas Dataframe
import pandas as pd
response_dict = response.json()
dividends = pd.DataFrame(response_dict['historical'])

In [ ]:
# Define our preferred date range
start_year = 2013
end_year = 2022

end_year = end_year + 1

list(range(start_year, end_year))

# Data Transformation
if dividends.shape == (0, 0):  # Handle the case where the company never issued any dividend in the past
    dividends = pd.DataFrame({
        "year": list(range(start_year - 1, end_year + 1)),
        "adjDividend": [0.0] * len(list(range(start_year - 1, end_year + 1)))  # We are obtaining 2 more years' data
    })
else:
    # Extract year data from the date column
    dividends['year'] = pd.to_datetime(dividends['date']).dt.year
    # Aggregate the dividend paid by year
    dividends = dividends.groupby("year").agg({"adjDividend": "sum"}).reset_index()
    # Create a new DataFrame with all years from start to end - So that we don't omit years without dividends
    all_years = pd.DataFrame({'year': list(range(start_year - 1, end_year + 1))})
    # Merge the two DataFrames on the year column and fill missing values with 0.0
    dividends = all_years.merge(dividends, on='year', how='left').fillna(0.0)

In [ ]:
# Let's create our target variable
import numpy as np

dividends['next_year_dividend'] = dividends['adjDividend'].shift(-1)

conditions = [
    dividends['adjDividend'] <= dividends['next_year_dividend'],
    dividends['adjDividend'] > dividends['next_year_dividend']
]

choices = ['constant/increased', 'decreased']

# Create the target column 'dps_change' based on the conditions
dividends['dps_change_next_year'] = np.select(conditions, choices, default=np.nan)

In [ ]:
# We can also create a predictor here - dps change from last year
dividends['last_year_dividend'] = dividends['adjDividend'].shift(1)
dividends['dps_growth'] = dividends['adjDividend'] - dividends['last_year_dividend']

In [ ]:
# Another predictor that we can create is dividend change as a percentage
dividends['dps_growth_rate'] = np.where(
    (dividends['last_year_dividend'] == 0) & (dividends['adjDividend'] == 0),
    0,  # If both are 0 then change is 0
    np.where(
        dividends['last_year_dividend'] != 0,
        ((dividends['adjDividend'] / dividends['last_year_dividend']) - 1) * 100,
        999  # If last year dividend is 0 then return 999
    )
)

# Remove the first last year since they will be NaN
dividends = dividends.loc[(dividends['year'] >= start_year) & (dividends['year'] <= end_year - 1)]
# Only keep the columns that we need
dividends = dividends[["year", "adjDividend", "dps_growth", "dps_growth_rate", "dps_change_next_year"]]

In [ ]:
# Engineer some other predictors
predictors = pd.DataFrame({"year": list(range(start_year - 1, end_year))})  # We include one more year before
                                                                            # the first year to calculate changes


# Let's include the Company's Industry and sector data
import yfinance as yf
company_data_raw = yf.Ticker(company_tick)
company_data = company_data_raw.info
predictors["industry"] = company_data['industry']
predictors["sector"] = company_data['sector']

In [ ]:
# Let's add some Macroeconomics indicators: the annualized Federal Interest Rate
API_KEY_FRED = os.environ.get('API_KEY_FRED')  # Retrieve the value of the environment variable 'API_KEY_FMP'

url = f'https://api.stlouisfed.org/fred/series/observations?series_id=FEDFUNDS&' \
      f'api_key={API_KEY_FRED}&' \
      f'file_type=json&' \
      f'observation_start={str(start_year - 1) + "-01-01"}&observation_end={str(end_year - 1) + "-12-31"}&' \
      f'frequency=a'

response = requests.get(url)
fed_interest_rates = pd.DataFrame(response.json()['observations'])
predictors['interestRate'] = fed_interest_rates['value'].astype("float64")

In [ ]:
# Now, lets add some other key financial ratios
num_of_years = 2024 - start_year + 1

response = requests.get(f"{BASE_URL}/ratios/{company_tick}?limit={num_of_years}&apikey={API_KEY_FMP}")

# Check if all year's data is available
data_length = len(response.json())
if data_length != num_of_years:
    print(f"Company {company_tick} data is not available")

financial_ratios = pd.DataFrame(response.json()).iloc[:, :].sort_values("date", ascending=True).reset_index(drop=True)
financial_ratios.info()
predictors.info()
financial_ratios['calendarYear'] = financial_ratios['calendarYear'].astype('int64')
predictors = predictors.merge(financial_ratios, left_on='year', right_on='calendarYear', how='left').fillna(0.0)

# Drop unnecessary columns
predictors.drop(["date", "calendarYear", "period"], axis="columns", inplace=True)


In [ ]:
def calculate_change(df, feature_name):
    percentage_change = df[feature_name].pct_change() * 100
    # Create new column name
    new_col_name = f"{feature_name}_percentage_change"
    # Find the index position of the original predictor column
    original_col_position = df.columns.get_loc(feature_name)
    # Insert the new column right after the original predictor column
    predictors.insert(original_col_position + 1, new_col_name, percentage_change)


feature_list = list(predictors.columns)

feature_list.remove('year')
feature_list.remove('industry')
feature_list.remove('sector')
feature_list.remove('symbol')



for feature in feature_list:
    calculate_change(predictors, feature)



# Replacing inf and NaN values
predictors.replace([float('inf'), float('-inf')], 999, inplace=True)
predictors.fillna(0, inplace=True)

In [ ]:
# Combine dividend data with other predictors
dataset = pd.merge(dividends, predictors, left_on='year', right_on='year', how='left')


# Move target to the end of the dataset for good practice
feature_list = list(dataset.columns)
feature_list.append('dps_change_next_year')
feature_list.remove('dps_change_next_year')
dataset = dataset[feature_list]